In [1]:
import pandas as pd
import time
import numpy as np
from scipy import sparse
import warnings
from implicit_markov_chain import ImplicitMarkovChain
from model_helper import save_model
warnings.filterwarnings('ignore')

# Load the data
print("Loading MovieLens 20M data...")
# selected_users = pd.read_csv('data/users_selection.csv')
ratings = pd.read_csv('data/filtered_high_ratings_20251114_120514.csv')

# print(f"Movies: {movies.shape}")
print(f"Ratings: {ratings.shape}")
print("\nFirst few rows of ratings:")
print(ratings.head())
# print("\nRating distribution:")
# print(ratings['rating'].value_counts().sort_index())

Loading MovieLens 20M data...
Ratings: (10246362, 4)

First few rows of ratings:
   userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819
3       1       47     3.5  1112484727
4       1       50     3.5  1112484580


# Loop

In [29]:
leave_k_out = 100
n_users = 1000

all_users = ratings['userId'].unique()

iterations = 100
max_recall, max_prec = 0, 0

for iter in range(iterations):
    print(f"{iter+1}/{iterations}")
    selected_user_ids = np.random.choice(all_users, size=n_users, replace=False)

    # print(f"Selected {len(selected_user_ids)} unique users")

    filtered_ratings = ratings[ratings['userId'].isin(selected_user_ids)]
    # print(f"Original ratings shape: {ratings.shape}")
    # print(f"Filtered ratings shape: {filtered_ratings.shape}")

    filtered_ratings_sorted = filtered_ratings.sort_values(['userId', 'timestamp'])

    train_ratings = filtered_ratings_sorted.groupby('userId').apply(lambda x: x.iloc[:-leave_k_out] if len(x) > leave_k_out else pd.DataFrame()).reset_index(drop=True)
    test_ratings = filtered_ratings_sorted.groupby('userId').tail(leave_k_out)

    print(f"Training set: {train_ratings.shape[0]:,} ratings")
    print(f"Test set: {test_ratings.shape[0]:,} ratings (Leave-{leave_k_out}-out)")

    print("Training implicit feedback Markov chain...")
    model = ImplicitMarkovChain(alpha=2.0)
    start = time.time()
    model.build_transition_matrix_efficient(train_ratings, sample_fraction=1)
    print(f"{time.time() - start:.4f} seconds")

    K = 50
    total_recall = 0
    users_not_considered = 0
    recalls = []
    total_precision = 0
    user_recommendations = {}
    train_ratings_sorted = train_ratings.sort_values(['userId', 'timestamp'])

    for user_id in selected_user_ids:
        user_data = train_ratings_sorted[train_ratings_sorted['userId'] == user_id]
        user_training_samples = len(user_data)
        
        # Only consider last 5 samples
        recent_movies = user_data['movieId'].tail(1).tolist()
        
        candidate_scores = {}
        
        # Process only the last 5 movies
        for movie_id in recent_movies:
            probabilities = model.transition_matrix[model.movie_to_idx[movie_id]]
            top_indices = np.argsort(probabilities)[-(K):][::-1]
            
            for i in top_indices:
                candidate_movie_id = model.idx_to_movie[i]
                candidate_prob = probabilities[i]
                
                # Skip if user has already rated this movie
                if candidate_movie_id in user_data['movieId'].values:
                    continue
                    
                # Keep the highest probability for each movie
                if candidate_movie_id not in candidate_scores:
                    candidate_scores[candidate_movie_id] = candidate_prob
                elif candidate_prob > candidate_scores[candidate_movie_id]:
                    candidate_scores[candidate_movie_id] = candidate_prob
        
        # Get top K movies from candidate scores
        top_movies = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)[:K]
        
        if len(top_movies) < K:
            users_not_considered += 1
            # print(f"userid: {user_id} - Insufficient recommendations: {len(top_movies)}/{K} - Skipping")
            continue
            
        candidate_movies = [movie_id for movie_id, score in top_movies]
        recommended_movies = set(candidate_movies)
        user_test_ratings = test_ratings[test_ratings['userId'] == user_id]
        relevant_movies = set(user_test_ratings['movieId'].tolist())
        
        # Validation
        hits = relevant_movies.intersection(recommended_movies)
        
        user_recall = 0
        if len(relevant_movies) > 0:
            user_recall = len(hits) / len(relevant_movies)
            total_recall += user_recall    
        recalls.append(user_recall)
        
        if len(recommended_movies) > 0:
            user_precision = len(hits) / len(recommended_movies)
            total_precision += user_precision

        # print(f"userid: {user_id} - recent_movies: {len(recent_movies)} - recommendations: {len(top_movies)} - test_data: {len(relevant_movies)} - hits: {len(hits)} - recall: {user_recall:.4f}")

    # print(f"total users: {len(selected_user_ids)} - users not consid. {users_not_considered} - total_recall: {total_recall}")

    if (len(selected_user_ids) - users_not_considered) > 0:
        avg_recall = total_recall / (len(selected_user_ids) - users_not_considered)
        avg_precision = total_precision / (len(selected_user_ids) - users_not_considered)
    else:
        avg_recall = 0
        avg_precision = 0

    max_recall = max(max_recall, avg_recall)
    max_prec = max(max_prec, avg_precision)
    print(f"Recall@{K}: {avg_recall:.4f} - max: {max_recall:.4f}")
    print(f"Precision@{K}: {avg_precision:.4f} - max: {max_prec:.4f}")


1/100
Training set: 57,449 ratings
Test set: 76,045 ratings (Leave-100-out)
Training implicit feedback Markov chain...
Processing user ratings...
User Groups: 423


100%|██████████| 423/423 [00:14<00:00, 29.41it/s]


16.1827 seconds
Recall@50: 0.1778 - max: 0.1778
Precision@50: 0.3557 - max: 0.3557
2/100
Training set: 59,767 ratings
Test set: 77,501 ratings (Leave-100-out)
Training implicit feedback Markov chain...
Processing user ratings...
User Groups: 441


100%|██████████| 441/441 [00:15<00:00, 29.23it/s]


17.1506 seconds
Recall@50: 0.1700 - max: 0.1778
Precision@50: 0.3400 - max: 0.3557
3/100
Training set: 47,312 ratings
Test set: 74,921 ratings (Leave-100-out)
Training implicit feedback Markov chain...
Processing user ratings...
User Groups: 406


...

23.2665 seconds
Recall@50: 0.1810 - max: 0.2115
Precision@50: 0.3620 - max: 0.4230
99/100
Training set: 52,893 ratings
Test set: 75,322 ratings (Leave-100-out)
Training implicit feedback Markov chain...
Processing user ratings...
User Groups: 409


100%|██████████| 409/409 [00:13<00:00, 30.78it/s]


14.9486 seconds
Recall@50: 0.1852 - max: 0.2115
Precision@50: 0.3704 - max: 0.4230
100/100
Training set: 53,746 ratings
Test set: 76,134 ratings (Leave-100-out)
Training implicit feedback Markov chain...
Processing user ratings...
User Groups: 430


100%|██████████| 430/430 [00:13<00:00, 32.01it/s]


16.3158 seconds
Recall@50: 0.1858 - max: 0.2115
Precision@50: 0.3717 - max: 0.4230


In [20]:
leave_k_out = 100
n_users = 10000

# selected_user_ids = np.random.choice(selected_users['x'].unique(), 1000, replace=False)
all_users = ratings['userId'].unique()
selected_user_ids = np.random.choice(all_users, size=n_users, replace=False)
print(f"Selected {len(selected_user_ids)} unique users")

filtered_ratings = ratings[ratings['userId'].isin(selected_user_ids)]
print(f"Original ratings shape: {ratings.shape}")
print(f"Filtered ratings shape: {filtered_ratings.shape}")

filtered_ratings_sorted = filtered_ratings.sort_values(['userId', 'timestamp'])

train_ratings = filtered_ratings_sorted.groupby('userId').apply(lambda x: x.iloc[:-leave_k_out] if len(x) > leave_k_out else pd.DataFrame()).reset_index(drop=True)
test_ratings = filtered_ratings_sorted.groupby('userId').tail(leave_k_out)

print(f"Training set: {train_ratings.shape[0]:,} ratings")
print(f"Test set: {test_ratings.shape[0]:,} ratings")

print("Training implicit feedback Markov chain...")
model = ImplicitMarkovChain(alpha=2.0)
start = time.time()
model.build_transition_matrix_efficient(train_ratings, sample_fraction=1)
print(f"{time.time() - start:.4f} seconds")

Selected 10000 unique users
Original ratings shape: (10246362, 4)
Filtered ratings shape: (1307853, 4)
Training set: 542,327 ratings
Test set: 765,526 ratings
Training implicit feedback Markov chain...
Processing user ratings...
User Groups: 4234


100%|██████████| 4234/4234 [04:05<00:00, 17.25it/s]


253.0982 seconds


In [ ]:
# from curses import window

K = 50
total_recall = 0
users_not_considered = 0
recalls = []
total_precision = 0
user_recommendations = {}
train_ratings_sorted = train_ratings.sort_values(['userId', 'timestamp'])

for user_id in selected_user_ids:
    user_data = train_ratings_sorted[train_ratings_sorted['userId'] == user_id]
    user_training_samples = len(user_data)
    
    candidate_scores = {}
    
    window_size = 0
    while len(candidate_scores) < K and window_size < len(user_data):
        # print(window_size)
        window_size += K
        additional_movies = user_data['movieId'].tail(window_size).head(K).tolist()

        
        for movie_id in additional_movies:
            probabilities = model.transition_matrix[model.movie_to_idx[movie_id]]
            top_indices = np.argsort(probabilities)[-(K):][::-1]
            
            movies_added = 0
            for i in top_indices:
                # print(f"movies added: {movies_added}")
                # if movies_added >= K:
                #     break
                    
                candidate_movie_id = model.idx_to_movie[i]
                candidate_prob = probabilities[i]
                
                # Skip if user has already rated this movie
                if candidate_movie_id in user_data['movieId'].values:
                    continue
                    
                # Keep the highest probability for each movie
                if candidate_movie_id not in candidate_scores:
                    candidate_scores[candidate_movie_id] = candidate_prob
                    movies_added += 1
                elif candidate_prob > candidate_scores[candidate_movie_id]:
                    candidate_scores[candidate_movie_id] = candidate_prob
                    
            # if len(candidate_scores) >= K:
            #     break


    top_movies = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)[:K]
    if len(top_movies) < K:
        users_not_considered += 1
        continue
    candidate_movies = [movie_id for movie_id, score in top_movies]
    recommended_movies = set(candidate_movies)
    user_test_ratings = test_ratings[test_ratings['userId'] == user_id]
    relevant_movies = set(user_test_ratings['movieId'].tolist())
    
    # validation
    hits = relevant_movies.intersection(recommended_movies)
    
    user_recall = 0
    if len(relevant_movies) > 0:
        user_recall = len(hits) / len(relevant_movies)
        total_recall += user_recall    
    recalls.append(user_recall)
    print(f"userid: {user_id} - len(top_movies): {len(top_movies)} - training_data: {len(user_data)} - test_data: {len(relevant_movies)} - window: {window_size} - hits: {len(hits)} recall: {user_recall}")

    if len(recommended_movies) > 0:
        user_precision = len(hits) / len(recommended_movies)
        total_precision += user_precision


print(f"total users: {len(selected_user_ids)} - users not consid. {users_not_considered} - total_recall: {total_recall}")
avg_recall = total_recall / (len(selected_user_ids)-users_not_considered)
avg_precision = total_precision / ((len(selected_user_ids)-users_not_considered)*K)

print("\n=== Final Results ===")
# print(f"Total users processed: {len(selected_user_ids)}")
print(f"Recall@{K}: {avg_recall:.4f}")
# print(f"Precision@{K}: {avg_precision:.4f}")

userid: 117305 - len(top_movies): 50 - training_data: 81 - test_data: 100 - window: 10 - hits: 14 recall: 0.14
userid: 28943 - len(top_movies): 50 - training_data: 45 - test_data: 100 - window: 10 - hits: 19 recall: 0.19
userid: 23301 - len(top_movies): 50 - training_data: 17 - test_data: 100 - window: 10 - hits: 34 recall: 0.34
userid: 67749 - len(top_movies): 50 - training_data: 206 - test_data: 100 - window: 10 - hits: 8 recall: 0.08
userid: 96468 - len(top_movies): 50 - training_data: 87 - test_data: 100 - window: 10 - hits: 17 recall: 0.17
userid: 67629 - len(top_movies): 50 - training_data: 148 - test_data: 100 - window: 10 - hits: 8 recall: 0.08
userid: 45958 - len(top_movies): 50 - training_data: 102 - test_data: 100 - window: 10 - hits: 3 recall: 0.03
userid: 58606 - len(top_movies): 50 - training_data: 6 - test_data: 100 - window: 10 - hits: 33 recall: 0.33
userid: 122589 - len(top_movies): 50 - training_data: 18 - test_data: 100 - window: 10 - hits: 37 recall: 0.37
userid: 1

In [27]:
K = 50
total_recall = 0
users_not_considered = 0
recalls = []
total_precision = 0
user_recommendations = {}
train_ratings_sorted = train_ratings.sort_values(['userId', 'timestamp'])

for user_id in selected_user_ids:
    user_data = train_ratings_sorted[train_ratings_sorted['userId'] == user_id]
    user_training_samples = len(user_data)
    
    # Only consider last 5 samples
    recent_movies = user_data['movieId'].tail(1).tolist()
    
    candidate_scores = {}
    
    # Process only the last 5 movies
    for movie_id in recent_movies:
        probabilities = model.transition_matrix[model.movie_to_idx[movie_id]]
        top_indices = np.argsort(probabilities)[-(K):][::-1]
        
        for i in top_indices:
            candidate_movie_id = model.idx_to_movie[i]
            candidate_prob = probabilities[i]
            
            # Skip if user has already rated this movie
            if candidate_movie_id in user_data['movieId'].values:
                continue
                
            # Keep the highest probability for each movie
            if candidate_movie_id not in candidate_scores:
                candidate_scores[candidate_movie_id] = candidate_prob
            elif candidate_prob > candidate_scores[candidate_movie_id]:
                candidate_scores[candidate_movie_id] = candidate_prob
    
    # Get top K movies from candidate scores
    top_movies = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)[:K]
    
    if len(top_movies) < K:
        users_not_considered += 1
        # print(f"userid: {user_id} - Insufficient recommendations: {len(top_movies)}/{K} - Skipping")
        continue
        
    candidate_movies = [movie_id for movie_id, score in top_movies]
    recommended_movies = set(candidate_movies)
    user_test_ratings = test_ratings[test_ratings['userId'] == user_id]
    relevant_movies = set(user_test_ratings['movieId'].tolist())
    
    # Validation
    hits = relevant_movies.intersection(recommended_movies)
    
    user_recall = 0
    if len(relevant_movies) > 0:
        user_recall = len(hits) / len(relevant_movies)
        total_recall += user_recall    
    recalls.append(user_recall)
    
    if len(recommended_movies) > 0:
        user_precision = len(hits) / len(recommended_movies)
        total_precision += user_precision

    print(f"userid: {user_id} - recent_movies: {len(recent_movies)} - recommendations: {len(top_movies)} - test_data: {len(relevant_movies)} - hits: {len(hits)} - recall: {user_recall:.4f}")

print(f"total users: {len(selected_user_ids)} - users not consid. {users_not_considered} - total_recall: {total_recall}")

if (len(selected_user_ids) - users_not_considered) > 0:
    avg_recall = total_recall / (len(selected_user_ids) - users_not_considered)
    avg_precision = total_precision / (len(selected_user_ids) - users_not_considered)
else:
    avg_recall = 0
    avg_precision = 0

print("\n=== Final Results ===")
print(f"Recall@{K}: {avg_recall:.4f}")
print(f"Precision@{K}: {avg_precision:.4f}")

userid: 58606 - recent_movies: 1 - recommendations: 50 - test_data: 100 - hits: 34 - recall: 0.3400
userid: 137151 - recent_movies: 1 - recommendations: 50 - test_data: 100 - hits: 3 - recall: 0.0300
userid: 116479 - recent_movies: 1 - recommendations: 50 - test_data: 100 - hits: 15 - recall: 0.1500
userid: 47743 - recent_movies: 1 - recommendations: 50 - test_data: 100 - hits: 18 - recall: 0.1800
userid: 58327 - recent_movies: 1 - recommendations: 50 - test_data: 100 - hits: 18 - recall: 0.1800
userid: 123958 - recent_movies: 1 - recommendations: 50 - test_data: 100 - hits: 19 - recall: 0.1900
userid: 122728 - recent_movies: 1 - recommendations: 50 - test_data: 100 - hits: 17 - recall: 0.1700
userid: 80029 - recent_movies: 1 - recommendations: 50 - test_data: 100 - hits: 23 - recall: 0.2300
userid: 111335 - recent_movies: 1 - recommendations: 50 - test_data: 100 - hits: 18 - recall: 0.1800
userid: 55021 - recent_movies: 1 - recommendations: 50 - test_data: 100 - hits: 30 - recall: 0.3